# Assignment 2 for FIT5212, Semester 1

**Student Name:** Michelle Fong

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162979 sha256=b71799cabd16682641cc9b244cab83806904264de541ba67b12d2a0a17f2a037
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD, KNNBasic
from surprise.model_selection import cross_validate, train_test_split

from surprise import NMF

from surprise.model_selection import GridSearchCV
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')

# Task 1:  Recommendation System

This part is to build a recommendation system predicting user rating using collaborativ filtering and various models. The following were considered when determinating the optimal model:
1. Voting & helpful_vote
2. Rounding of rating prediction
3. Training model algorithm

Various model are fitted and compared against each other. The results are measured by using metrics: RMSE, MAE. Further discussion would be held in the pdf report.

## Data Loading

In [8]:
path = '/content/drive/MyDrive/Colab Notebooks/fit5212/asg2/'

# path = '/content/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
train.head(5)

,user_id,product_id,product_name,rating,votes,helpful_votes,ID
0,1813,154533,Beautiful Thing,5,10,8,0
1,1944,192838,Almost Famous,5,4,2,1
2,534,202590,A Clockwork Orange,5,5,5,2
3,1811,140456,Great Expectations (Wordsworth Classics),4,1,0,3
4,102,154278,Phenomenon,5,0,0,4


In [5]:
print(len(train.user_id.unique()),
len(train.product_id.unique()))

2000 201325


In [6]:
print(train.shape, test.shape)

(745889, 7) (223553, 4)


In [9]:
test.head(5)

,ID,user_id,product_id,product_name
0,0,1916,185507,Maria [Australia]
1,1,1759,143430,"The Mitchells: Five for Victory (Van Stockum, ..."
2,2,1980,155861,Superman Cartoons of Max & Dav
3,3,1502,71535,The Rainbow Fish Anniversary Edition
4,4,1255,13521,Confessions (Oxford World's Classics)


## Data exploring

In [8]:
n_users = train.user_id.unique().shape[0]
n_prod = train.product_id.unique().shape[0]
n_ratings = len(train)
avg_ratings_per_user = n_prod/n_users

print('Number of unique users: ', n_users)
print('Number of unique products: ', n_prod)
print('Number of total ratings: ', n_ratings)
print('Average number of ratings per user: ', avg_ratings_per_user)

Number of unique users:  2000
Number of unique products:  201325
Number of total ratings:  745889
Average number of ratings per user:  100.6625


## Memory Based

In [ ]:
# ratings_matrix = pd.pivot_table(train, values='rating', index='user_id', columns='product_id')
# normalized_ratings_matrix = ratings_matrix.subtract(ratings_matrix.mean(axis=1), axis=0)
# similarity_matrix = ratings_matrix.T.corr()

## Model 1: Whole Training set

In [9]:
reader = Reader(rating_scale=(1,5))

In [10]:
train_data = Dataset.load_from_df(train[['user_id', 'product_id', 'rating']],
                                  reader)

In [11]:
# memory-based
knn = KNNBasic()
cross_validate(knn, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0188  1.0145  1.0183  1.0179  1.0160  1.0171  0.0016  
MAE (testset)     0.7264  0.7258  0.7274  0.7273  0.7262  0.7266  0.0006  
Fit time          1.40    1.75    1.31    1.47    1.61    1.51    0.16    
Test time         14.43   13.04   11.05   6.00    7.75    10.45   3.16    


{'test_rmse': array([1.0188374 , 1.01447819, 1.01825476, 1.01792406, 1.01601932]),
 'test_mae': array([0.72638538, 0.72579426, 0.72741979, 0.72729092, 0.72620052]),
 'fit_time': (1.3991310596466064,
  1.7509641647338867,
  1.3098170757293701,
  1.4675405025482178,
  1.6076343059539795),
 'test_time': (14.427459955215454,
  13.038833618164062,
  11.050644159317017,
  6.004701852798462,
  7.7460103034973145)}

In [12]:
# model-based
svd = SVD()
cross_validate(svd, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9390  0.9397  0.9402  0.9379  0.9346  0.9383  0.0020  
MAE (testset)     0.6933  0.6932  0.6915  0.6894  0.6950  0.6925  0.0019  
Fit time          19.86   21.23   25.40   29.55   24.94   24.20   3.41    
Test time         1.68    1.67    2.47    3.84    1.56    2.24    0.86    


{'test_rmse': array([0.93898704, 0.93974167, 0.94019535, 0.93791259, 0.93456588]),
 'test_mae': array([0.69332381, 0.69323156, 0.69148317, 0.68941135, 0.69495942]),
 'fit_time': (19.86436104774475,
  21.227962017059326,
  25.396515130996704,
  29.551833152770996,
  24.937644958496094),
 'test_time': (1.679917573928833,
  1.6690199375152588,
  2.4683501720428467,
  3.8422365188598633,
  1.5638434886932373)}

In [13]:
# predict function iterating each row
def predict_ratings(test_df, model):
    predictions = []
    for index, row in test_df.iterrows():
        user_id = row['user_id']
        product_id = row['product_id']
        prediction = model.predict(user_id, product_id).est
        predictions.append((row['ID'], prediction))
    return predictions

# rounded predict
def predict_ratings_rounded(test_df, model):
    predictions = []
    for index, row in test_df.iterrows():
        user_id = row['user_id']
        product_id = row['product_id']
        prediction = round(model.predict(user_id, product_id).est)
        predictions.append((row['ID'], prediction))
    return predictions

In [29]:
model_1_test_pred_knn = predict_ratings(test, knn)
result_df = pd.DataFrame(model_1_test_pred_knn, columns=['ID', 'rating'])
result_df.to_csv('M1_output_knn.csv', index=False)
# public score: 1.05338 (output 11)

In [30]:
model_1_test_pred_svd = predict_ratings(test, svd)
result_df = pd.DataFrame(model_1_test_pred_svd, columns=['ID', 'rating'])
result_df.to_csv('M1_output_svd.csv', index=False)
# public score: 0.96668 (output 12)


## Model 2: SVD with rounding

In [31]:
M2_test_pred = predict_ratings_rounded(test, svd)
result_df = pd.DataFrame(M2_test_pred, columns=['ID', 'rating'])
result_df.to_csv('M2_output.csv', index=False)
# public score: 1.02399 (output 18)


## Model 3: Partial Training set: Voting & Helpful_vote

In [10]:
train.describe()

,user_id,product_id,rating,votes,helpful_votes,ID
count,745889.000000,745889.000000,745889.000000,745889.000000,745889.000000,745889.000000
mean,1143.427653,116563.109093,4.238671,7.408667,5.381212,372944.000000
std,625.686509,68180.221826,1.072359,16.949467,13.089394,215319.751805
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,590.000000,54561.000000,4.000000,1.000000,1.000000,186472.000000
50%,1227.000000,119997.000000,5.000000,4.000000,2.000000,372944.000000
75%,1748.000000,182973.000000,5.000000,8.000000,6.000000,559416.000000
max,1999.000000,219858.000000,5.000000,3740.000000,2537.000000,745888.000000


In [33]:
train_filtered = train[(train['votes'] > train['votes'].mean()) & (train['helpful_votes'] > train['helpful_votes'].mean())]
train_filtered.shape


(174695, 7)

In [34]:
train_filtered_data = Dataset.load_from_df(train_filtered[['user_id', 'product_id', 'rating']],
                                  reader)

In [35]:
knn = KNNBasic()
cross_validate(knn, train_filtered_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0261  1.0202  1.0303  1.0422  1.0326  1.0303  0.0073  
MAE (testset)     0.7192  0.7169  0.7214  0.7307  0.7223  0.7221  0.0047  
Fit time          0.21    0.22    0.22    0.27    0.33    0.25    0.05    
Test time         0.52    0.51    0.76    0.73    2.68    1.04    0.83    


{'test_rmse': array([1.02613967, 1.02016284, 1.03032793, 1.04222647, 1.03255754]),
 'test_mae': array([0.71917874, 0.71694813, 0.7214462 , 0.73070663, 0.72225086]),
 'fit_time': (0.2097325325012207,
  0.21958327293395996,
  0.2237405776977539,
  0.2691066265106201,
  0.33263230323791504),
 'test_time': (0.5165715217590332,
  0.5084643363952637,
  0.764794111251831,
  0.7266788482666016,
  2.6832785606384277)}

In [36]:
svd = SVD()
cross_validate(svd, train_filtered_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9504  0.9543  0.9487  0.9402  0.9579  0.9503  0.0060  
MAE (testset)     0.7002  0.6978  0.6963  0.6900  0.7007  0.6970  0.0038  
Fit time          3.88    4.96    4.56    4.19    5.08    4.54    0.45    
Test time         0.30    1.24    0.24    0.26    0.40    0.49    0.38    


{'test_rmse': array([0.95035912, 0.95431184, 0.94873241, 0.94023051, 0.95786364]),
 'test_mae': array([0.70019186, 0.6977775 , 0.69632944, 0.69002004, 0.7006607 ]),
 'fit_time': (3.8815953731536865,
  4.962086200714111,
  4.564722299575806,
  4.187767744064331,
  5.080131769180298),
 'test_time': (0.296588659286499,
  1.2432236671447754,
  0.23912906646728516,
  0.26492905616760254,
  0.39843034744262695)}

In [37]:
trainset_filtered = train_filtered_data.build_full_trainset()
svd.fit(trainset_filtered)

M3_test_pred = predict_ratings(test, svd)

# Output the results in the desired form
result_df = pd.DataFrame(M3_test_pred, columns=['ID', 'rating'])
result_df.to_csv('M3_output.csv', index=False)
# public score: 1.03176 (output 10)

In [38]:
# rounding
# predict
def predict_ratings_rounded(test_df, model):
    predictions = []
    for index, row in test_df.iterrows():
        user_id = row['user_id']
        product_id = row['product_id']
        prediction = round(model.predict(user_id, product_id).est)
        predictions.append((row['ID'], prediction))
    return predictions


M3_test_pred_round = predict_ratings_rounded(test, svd)
# Output the results in the desired form
result_df = pd.DataFrame(M3_test_pred_round, columns=['ID', 'rating'])
result_df.to_csv('M3_output_rounded.csv', index=False)
# public score: 1.08095 (output 4)

## Model 4: baseline model using ALS

In [11]:
train.head(5)

,user_id,product_id,product_name,rating,votes,helpful_votes,ID
0,1813,154533,Beautiful Thing,5,10,8,0
1,1944,192838,Almost Famous,5,4,2,1
2,534,202590,A Clockwork Orange,5,5,5,2
3,1811,140456,Great Expectations (Wordsworth Classics),4,1,0,3
4,102,154278,Phenomenon,5,0,0,4


In [39]:
train_data = Dataset.load_from_df(train[['user_id', 'product_id', 'rating']],
                                  reader)

In [40]:
from surprise import BaselineOnly
bsl_options = {'method': 'als'}
bias_baseline = BaselineOnly(bsl_options)

cross_validate(bias_baseline, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9379  0.9366  0.9408  0.9367  0.9405  0.9385  0.0018  
MAE (testset)     0.6989  0.6994  0.7013  0.6987  0.7014  0.6999  0.0012  
Fit time          6.14    7.10    6.58    7.05    6.69    6.71    0.35    
Test time         4.38    2.29    1.78    2.29    1.63    2.48    0.99    


{'test_rmse': array([0.93792496, 0.93661354, 0.94081478, 0.93669841, 0.94054825]),
 'test_mae': array([0.69887722, 0.69937291, 0.70127262, 0.69873837, 0.70136852]),
 'fit_time': (6.142350196838379,
  7.101520776748657,
  6.577631950378418,
  7.048041343688965,
  6.692445516586304),
 'test_time': (4.381750583648682,
  2.2919414043426514,
  1.7848269939422607,
  2.290771961212158,
  1.6339161396026611)}

In [41]:
trainset = train_data.build_full_trainset()
bias_baseline.fit(trainset)

M4_test_pred = predict_ratings(test, bias_baseline)

# Output the results in the desired form
result_df = pd.DataFrame(M4_test_pred, columns=['ID', 'rating'])
result_df.to_csv('M4_output.csv', index=False)
# public score: 0.96

Estimating biases using als...


## Model 5: NMF

In [23]:
nmf = NMF()
cross_validate(nmf, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0675  1.0575  1.0640  1.0770  1.0870  1.0706  0.0103  
MAE (testset)     0.8207  0.8094  0.8168  0.8307  0.8479  0.8251  0.0133  
Fit time          50.15   51.08   47.49   50.44   49.14   49.66   1.25    
Test time         2.31    1.01    2.82    2.88    1.51    2.11    0.74    


{'test_rmse': array([1.06746257, 1.05752429, 1.06400524, 1.0769567 , 1.08704243]),
 'test_mae': array([0.82074649, 0.80944905, 0.81679877, 0.83070911, 0.8479026 ]),
 'fit_time': (50.14686369895935,
  51.07991981506348,
  47.493454933166504,
  50.43664574623108,
  49.141982555389404),
 'test_time': (2.3110432624816895,
  1.0135891437530518,
  2.81864857673645,
  2.884467363357544,
  1.5112383365631104)}

In [42]:
#using nmf
trainset = train_data.build_full_trainset()
nmf.fit(trainset)

In [43]:
M5_test_pred = predict_ratings(test, nmf)

# Output the results in the desired form
result_df = pd.DataFrame(M5_test_pred, columns=['ID', 'rating'])
result_df.to_csv('M5_output.csv', index=False)
# public score: 1.07 (output 15)

## Model 6: SVD with params tuning

In [44]:
param_grid = {'n_factors': [50, 100, 150],
              'reg_all': [0.01, 0.02, 0.05],
              'lr_all': [0.005, 0.01, 0.02]}

grid_search = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
grid_search.fit(train_data)

best_params = grid_search.best_params['rmse']
best_score = grid_search.best_score['rmse']

print("Best Parameters:", best_params)
print("Best RMSE Score:", best_score)

Best Parameters: {'n_factors': 150, 'reg_all': 0.05, 'lr_all': 0.02}
Best RMSE Score: 0.9000574046864308


In [45]:
svd_model = SVD(**best_params)
svd_model.fit(trainset)
M6_test_pred = predict_ratings(test, svd_model)

# Output as df and export
result_df = pd.DataFrame(M6_test_pred, columns=['ID', 'rating'])
result_df.to_csv('M6_output.csv', index=False)
# public score: 0.91922 (output 16)

## Model 7: Diff way of making prediciton using SVD

In [46]:
from surprise import SVD
from surprise import Dataset, Reader
import pandas as pd

train_data = train[['user_id', 'product_id', 'rating']]
test_data = test[['user_id', 'product_id']]

reader = Reader(rating_scale=(1, 5))
trainset = Dataset.load_from_df(train_data, reader).build_full_trainset()

svd_model = SVD(n_factors=150, reg_all=0.05, lr_all=0.02)
svd_model.fit(trainset)

# # Train the SVD model
# svd_model = SVD(n_factors=100, reg_all=0.05, lr_all=0.02)
# svd_model.fit(trainset)


testset = list(zip(test_data['user_id'].values, test_data['product_id'].values))
M7_test_pred = [svd_model.predict(uid, iid) for uid, iid in testset]
predicted_ratings = [pred.est for pred in M7_test_pred]


result_df = pd.DataFrame({'ID': test_data.index, 'rating': predicted_ratings})
result_df.to_csv('M7_output.csv', index=False)
# public score: 0.91945 (output 17)

## Model 8: 2nd attemp tuning param of SVD

In [52]:
param_grid_2 = {'n_factors': [100, 150, 200],
              'reg_all': [0.02, 0.05, 0.07],
              'lr_all': [0.01, 0.03, 0.05]}
train_data = Dataset.load_from_df(train[['user_id', 'product_id', 'rating']],
                                  reader)
grid_search = GridSearchCV(SVD, param_grid_2, measures=['rmse', 'mae'], cv=5)
grid_search.fit(train_data)

best_params_2 = grid_search.best_params['rmse']
best_score_2 = grid_search.best_score['rmse']

In [53]:
print("Best Parameters:", best_params_2)
print("Best RMSE Score:", best_score_2)

Best Parameters: {'n_factors': 200, 'reg_all': 0.05, 'lr_all': 0.05}
Best RMSE Score: 0.8852759818256322


In [54]:
# prediciton with all user and product id
from surprise import SVD
from surprise import Dataset, Reader
import pandas as pd

train_data = train[['user_id', 'product_id', 'rating']]
test_data = test[['user_id', 'product_id']]

reader = Reader(rating_scale=(1, 5))
trainset = Dataset.load_from_df(train_data, reader).build_full_trainset()

svd_model = SVD(n_factors=200, reg_all=0.05, lr_all=0.05)
svd_model.fit(trainset)

# # Train the SVD model
# svd_model = SVD(n_factors=100, reg_all=0.05, lr_all=0.02)
# svd_model.fit(trainset)

testset = list(zip(test_data['user_id'].values, test_data['product_id'].values))
M8_test_pred = [svd_model.predict(uid, iid) for uid, iid in testset]

predicted_ratings = [pred.est for pred in M8_test_pred]


result_df = pd.DataFrame({'ID': test_data.index, 'rating': predicted_ratings})
result_df.to_csv('M8_output.csv', index=False)
# public score: 0.90429

## Model 9: NCF

In [55]:
# here is a handy function modified from fast.ai
def proc_col(col, train_col=None):
    """Encodes a pandas column with continous ids.
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids.
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["user_id", "product_id"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

import torch
import torch.nn as nn
import torch.nn.functional as F

class CollabFNet(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100, n_hidden=10):
        super(CollabFNet, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.lin1 = nn.Linear(emb_size*2, n_hidden)
        self.lin2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.1)

    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        x = F.relu(torch.cat([U, V], dim=1))
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        x = 4* torch.sigmoid(x) +1
        return x

def test_loss(model, val_data, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(val_data.user_id.values)
    items = torch.LongTensor(val_data.product_id.values)
    ratings = torch.FloatTensor(val_data.rating.values)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())
    print("rating: ",ratings)
    print("predictions:", y_hat.squeeze().detach().cpu().numpy())

def train_epocs(model, train_data, val_data, epochs=5, lr=0.01, wd=0.01, batch_size=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        for i in range(0, len(train_data), batch_size):
            users = torch.LongTensor(train_data.user_id.values[i:i+batch_size])
            items = torch.LongTensor(train_data.product_id.values[i:i+batch_size])
            ratings = torch.FloatTensor(train_data.rating.values[i:i+batch_size])

            optimizer.zero_grad()
            outputs = model(users, items)
            loss = criterion(outputs.squeeze(), ratings)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
        test_loss(model, val_data)

In [56]:
n_users = train.user_id.unique().shape[0]
n_prod = train.product_id.unique().shape[0]
n_ratings = len(train)
avg_ratings_per_user = n_prod/n_users

print('Number of unique users: ', n_users)
print('Number of unique movies: ', n_prod)
print('Number of total ratings: ', n_ratings)
print('Average number of ratings per user: ', avg_ratings_per_user)

Number of unique users:  2000
Number of unique movies:  201325
Number of total ratings:  745889
Average number of ratings per user:  100.6625


In [57]:
# Train-test split
np.random.seed(3)
msk = np.random.rand(len(train_filtered)) < 0.8
train_split = train_filtered[msk].copy()
val_split = train_filtered[~msk].copy()
df_train = encode_data(train_split)
df_val = encode_data(val_split, train_split)
test_encode = encode_data(test, train_split)

In [59]:
model = CollabFNet(num_users=len(df_train.user_id.unique()),
                   num_items=len(df_train.product_id.unique()))
# model.cuda()
train_epocs(model, df_train, df_val,lr=0.0005, wd=0.05)

Epoch 1/5, Loss: 1.6222
test loss 1.259 
rating:  tensor([5., 4., 5.,  ..., 5., 4., 5.])
predictions: [4.2205124 4.312767  4.235459  ... 4.3062296 4.378082  4.2652035]
Epoch 2/5, Loss: 1.6135
test loss 1.254 
rating:  tensor([5., 4., 5.,  ..., 5., 4., 5.])
predictions: [4.2539816 4.2599745 4.173725  ... 4.2999697 4.262755  4.225014 ]
Epoch 3/5, Loss: 1.6123
test loss 1.246 
rating:  tensor([5., 4., 5.,  ..., 5., 4., 5.])
predictions: [4.26      4.2571616 4.1912775 ... 4.25503   4.213743  4.227606 ]
Epoch 4/5, Loss: 1.6234
test loss 1.243 
rating:  tensor([5., 4., 5.,  ..., 5., 4., 5.])
predictions: [4.2608333 4.264618  4.2500257 ... 4.258134  4.2508187 4.255759 ]
Epoch 5/5, Loss: 1.6259
test loss 1.243 
rating:  tensor([5., 4., 5.,  ..., 5., 4., 5.])
predictions: [4.269029  4.270111  4.269558  ... 4.270029  4.2696004 4.269647 ]


In [60]:
test_encode = encode_data(test, train_split)
test_encode_cuda = torch.tensor(test_encode[['user_id', 'product_id']].values) #.cuda()

model.eval()

# Make predictions on the test dataset
with torch.no_grad():
    M9_test_predpredictions = model(test_encode_cuda[:, 0], test_encode_cuda[:, 1])


# M9_test_pred = predictions.cpu().numpy()

M9_prediction = M9_test_predpredictions.numpy()

results_df = pd.DataFrame({'ID': test_encode['ID'], 'rating': M9_prediction.flatten()})
print(results_df)


            ID    rating
2            2  4.269832
3            3  4.271633
4            4  4.271760
5            5  4.269818
7            7  4.269370
...        ...       ...
223548  223548  4.281526
223549  223549  4.269655
223550  223550  4.275832
223551  223551  4.268971
223552  223552  4.281719

[131822 rows x 2 columns]


In [61]:
# extract mising id
missing_ids = set(test['ID']) - set(test_encode['ID'])

# fill with 3
missing_df = pd.DataFrame({'ID': list(missing_ids), 'rating': [3] * len(missing_ids)})


results_df = pd.concat([results_df, missing_df]).sort_values(by='ID')
print(results_df)


            ID    rating
0            0  3.000000
1            1  3.000000
2            2  4.269832
3            3  4.271633
4            4  4.271760
...        ...       ...
223548  223548  4.281526
223549  223549  4.269655
223550  223550  4.275832
223551  223551  4.268971
223552  223552  4.281719

[223553 rows x 2 columns]


In [62]:
results_df.to_csv('M9_output.csv', index=False)
# public score: 1.36962 (output 17)

## Summary of all model result

In [4]:
summary_df = pd.DataFrame({
    'Model':['M1a', 'M1b',
             'M2',
             'M3a', 'M3b',
             'M4',
             'M5',
             'M6',
             'M7',
             'M8',
             'M9'],

    'Name':['KNN', 'SVD',
            'SVD with rounding',
            'SVD with filtered data', 'SVD with filtered data & rounded',
            'Baseline ALS',
            'NMF',
            'SVD with tuned param',
            'SVD with diff way of pred',
            'SVD with tuned param 2',
            'NCF'],

    'Public Score':[1.05338, 0.96668,
                    1.02153,
                    1.03176,1.08095,
                    0.96534,
                    1.08821,
                    0.91922,
                    0.91945,
                    0.90429,
                    1.34951]
})

summary_df.set_index('Model', inplace=True)
sorted_summary_df = summary_df.sort_values(by='Public Score')
sorted_summary_df

,Name,Public Score
Model,,
M8,SVD with tuned param 2,0.90429
M6,SVD with tuned param,0.91922
M7,SVD with diff way of pred,0.91945
M4,Baseline ALS,0.96534
M1b,SVD,0.96668
M2,SVD with rounding,1.02153
M3a,SVD with filtered data,1.03176
M1a,KNN,1.05338
M3b,SVD with filtered data & rounded,1.08095
